<a href="https://colab.research.google.com/github/FelipeTufaile/MixtureModels/blob/main/MixtureModels/tests/Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [550]:
import numpy as np
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt
from matplotlib.patches import Circle, Arc

In [551]:
def init_mixture(X, k, random_state):
  """
  Initializes the mixture model with random points as initial means and uniform assingments

  Args:
      X: (n, d) array holding the data

  Returns:
      mixture: the initialized gaussian mixture
      post: (n, K) array holding the soft counts for all components for all examples
  """
  # Setting seed
  np.random.seed(random_state)

  # Calculating the number of samples "n" in the dataset (X array)
  n, d = X.shape

  # Initializing the weight (mixing proportions) for each component (cluster)
  p = (np.ones(k) / k).reshape(-1,1)

  # Initialize the mean array with random points from X as initial means
  mu = X[np.random.choice(n, k, replace=False),:]

  # Initialize the variance array for each component (cluster)
  var = np.array([np.sum((X-mu[j,:])**2, axis=0) for j in range(k)])/(n-1)

  return mu, p, var

In [552]:
def GaussianProbability(X, mu, var, multivariate_normal=multivariate_normal.pdf):
    """
    The function calculates the probability of X belonging to the Gaussian distribution using the multivariate Gaussian PDF formula.

    Args:
        X (numpy.ndarray): Point with shape (1, d).
        mu (numpy.ndarray): Mean of the Gaussian distribution with shape (1, d).
        var (numpy.ndarray): Variance of the Gaussian distribution with shape (1, d).

    Returns:
        float: Probability of X belonging to the Gaussian distribution.
    """

    # Calculating probability
    probabilities = multivariate_normal(X, mean=mu, cov=np.diag(var)).T

    return probabilities


In [553]:
def estep(X, GMM, GaussianProbability=GaussianProbability):
    """
    E-step: Softly assigns each datapoint to a gaussian component

    Args:
        X: (n, d) array holding the data mixture: the current gaussian mixture

    Returns:
        np.ndarray: (n, K) array holding the soft counts for all components for all examples
        float: log-likelihood of the assignment
    """

    # Calculating the probability for each datapoint [i] belonging to each clusters [j]: probabilities has shape n x j
    probabilities = np.array([GaussianProbability(X=X, mu=GMM.mu[j,:], var=GMM.var[j,:]) for j in range(GMM.k)]).T

    # Calulating the likelihood for each each datapoint [i] belonging to each clusters [j]: likelihoods has shape n x 1
    likelihoods = probabilities@GMM.p

    # Calculating the posterior probabilities for each datapoint [i] belonging to each clusters [j]: post has shape n x j
    post = ((probabilities.T)*GMM.p).T/likelihoods

    # Calculating the sum of log_likelihoods
    sum_log_likelihood = np.sum(np.log(likelihoods))

    return post, sum_log_likelihood

In [554]:
def mstep(X, post):
  """
  M-step: Updates the gaussian mixture by maximizing the log-likelihood of the weighted dataset

  Args:
      X: (n, d) array holding the data
      post: (n, K) array holding the soft counts for all components for all examples

  Returns:
      GaussianMixture: the new gaussian mixture
  """

  # Calculating the sum of the posterior probabilities sum_p has shape j x 1
  sum_p = np.sum(post, axis=0).reshape(-1,1)

  # Updating mixture proportions: p has shape j x 1
  p = sum_p/X.shape[0]

  # Updating the center of each distribution (cluster)
  mu = post.T@X/sum_p

  # Updating the variance of each distribution (cluster)
  var = np.array([np.sum(post[:,j].reshape(-1,1)*((X-mu[j,:].reshape(1,-1))**2), axis=0) for j in range(post.shape[1])])/(X.shape[1]*sum_p)

  return mu, p, var


In [555]:
class GaussianMixture():
  """
  Tuple holding a gaussian mixture

  mu: np.ndarray  # (K, d) array - each row corresponds to a gaussian component mean
  var: np.ndarray  # (K, ) array - each row corresponds to the variance of a component
  p: np.ndarray  # (K, ) array = each row corresponds to the weight of a component
  """
  """
    Initializes the gaussian mixture model.

    Args:
        K (int): number of components
        seed (int): random seed

    Returns:
        mixture: the initialized gaussian mixture model
    """

  def __init__(
      self,
      k,
      max_iter=10000,
      tol=10**(-5),
      n_init=1,
      verbose=False,
      random_state=0
    ):

    self.k = k
    self.max_iter = max_iter
    self.tol = tol
    self.n_init = n_init
    self.verbose = verbose
    self.random_state = random_state

  def fit(self, X, init_mixture=init_mixture, estep=estep, mstep=mstep):

    # Initialize mixture
    self.mu, self.p, self.var = init_mixture(X, self.k, self.random_state)

    #return self

    # Initialize l_theta
    l_log_likelihood = []

    for i in range(self.max_iter):

      # Run E-Step
      post, log_likelihood = estep(X=X, GMM=self)

      # Run M-Step
      self.mu, self.p, self.var = mstep(X=X, post=post)

      # Update the list of log_likelihood
      l_log_likelihood.append(log_likelihood)

    return self, l_log_likelihood


In [573]:
samples1 = np.random.normal(np.array([10, 15]), np.array([1, 3]), (1000,2))
samples2 = np.random.normal(np.array([50, 70]), np.array([5, 8]), (1000,2))

X = np.concatenate([samples1, samples2])

# Initializing a Gaussian Mixture Model
GMM = GaussianMixture(k=2, max_iter=100, random_state=88)

# Fitting the data to the model and finding distributions
GMM, l_log_likelihood = GMM.fit(X=X)
#GMM = GMM.fit(X=X)

# Printing Gaussian Mixture Model parameters
for i, params in enumerate(zip(GMM.mu, GMM.var)):
  print(f"Center of distribution {str(i).zfill(3)} is {params[0]} and standard deviation is {params[1]**0.5}")

Center of distribution 000 is [50.04358905 70.29821307] and standard deviation is [3.43401233 5.6785979 ]
Center of distribution 001 is [10.01021005 14.91655592] and standard deviation is [0.69425842 2.08572533]
